# 🚀 Progetto finale: Gestione di un inventario avanzato

---
## Introduzione al progetto

In questo capitolo, metteremo insieme tutti i concetti imparati finora per creare un piccolo programma che gestisce un inventario di prodotti. Questa volta, useremo un approccio più professionale e modulare, sfruttando in particolare la **Programmazione Orientata agli Oggetti** per modellare il problema in modo più efficace.

Utilizzeremo:

- **Ambienti virtuali (`venv`)** per isolare le dipendenze del progetto.
- **Classi e Oggetti (OOP)** per rappresentare i prodotti e l'inventario stesso.
- **Concetti avanzati di OOP** come **ereditarietà**, **incapsulamento** e **polimorfismo**.
- **Librerie esterne** come `colorama` per migliorare l'interfaccia utente.
- **Strutture dati** (dizionari) per organizzare l'inventario all'interno di una classe.
- **Cicli (`while`, `for`)** per l'interazione con l'utente e l'elaborazione dei dati.
- **Gestione degli errori (`try-except`)** per rendere il programma robusto.
- **Input/Output da file** con il modulo `json` per salvare e caricare i dati in modo persistente.
- **Uso di un decorator** che ogni volta che viene eseguita un’azione (aggiunta, vendita, salvataggio, caricamento), registra l'azione file actions.log


> 📂 **[Clicca qui per vedere il codice del progetto](code/12/solutions)**


---
## Preparazione del Progetto con Ambienti Virtuali 💻

Prima di scrivere il codice, occorre creare un ambiente isolato per il nostro progetto. Questo ci assicura che le dipendenze non entrino in conflitto con altri progetti sul tuo sistema.

Aprire il terminale, navigare nella cartella del progetto ed eseguire i seguenti comandi:

1.  **Creare l'ambiente virtuale:**
    `python3 -m venv venv`

2.  **Attivare l'ambiente virtuale:**
    *Su macOS / Linux:* `source venv/bin/activate`
    *Su Windows:* `venv\Scripts\activate.bat`

3.  **Installare la libreria esterna `colorama`:**
    `pip install colorama`


---
## Struttura del Progetto e Gestione delle Dipendenze 📁

Per un'organizzazione chiara e professionale, il progetto è suddiviso in più file. Questo rende il codice più facile da leggere, testare e mantenere.

La struttura della cartella del progetto sarà la seguente:

```
inventory-manager/
├── venv/                   # Ambiente virtuale
├── inventory.json         # File di salvataggio dei dati
├── requirements.txt        # Dipendenze del progetto
├── main.py                 # Loop principale e interfaccia utente
├── models.py               # Classi per la modellazione dei prodotti
├── inventory_manager.py    # Classe per la gestione dell'inventario
└── test_inventory.py   # File di test per la logica del programma
```

### File `requirements.txt`

Questo file elenca le librerie esterne necessarie per il progetto.

```txt
colorama==0.4.6
```

Si possono installare tutte le dipendenze usando pip:

```bash
pip install -r requirements.txt
```

---
## Implementazione del Codice 💻

Di seguito il codice suddiviso nei rispettivi file. **Eseguire ogni cella in ordine per creare i file del progetto prima di eseguire `main.py` o i test.**

### `models.py`

Questa cella creerà il file `models.py` contenente le classi base e le sottoclassi dei prodotti.

In [7]:
%%writefile models.py
# models.py

from abc import ABC, abstractmethod

# Abstract base class for all products
class BaseItem(ABC):
    def __init__(self, name: str, price: float, quantity: int):
        self.name = name
        self.__price = price
        self.quantity = quantity

    # Abstract method that each subclass must implement
    @abstractmethod
    def __str__(self):
        pass
    
    # Method for encapsulating the price
    def get_price(self) -> float:
        return self.__price

    # Method to set the price, with validation
    def set_price(self, new_price: float):
        if new_price > 0:
            self.__price = new_price
        else:
            print("Price must be greater than 0.")
            
    def to_dict(self):
        return {'name': self.name, 'price': self.get_price(), 'quantity': self.quantity}
    
    @classmethod
    def from_dict(cls, data):
        return cls(name=data['name'], price=data['price'], quantity=data['quantity'])


# Subclass for food products
class FoodItem(BaseItem):
    def __init__(self, name: str, price: float, quantity: int, expiration_date: str):
        super().__init__(name, price, quantity)
        self.expiration_date = expiration_date
    
    def __str__(self):
        return f"Food Item: {self.name.capitalize()} | Price: {self.get_price():.2f}€ | Quantity: {self.quantity} | Expiration Date: {self.expiration_date}"

    def to_dict(self):
        data = super().to_dict()
        data['expiration_date'] = self.expiration_date
        return data

# Subclass for electronic products
class ElectronicItem(BaseItem):
    def __init__(self, name: str, price: float, quantity: int, warranty_years: int):
        super().__init__(name, price, quantity)
        self.warranty_years = warranty_years
        
    def __str__(self):
        return f"Electronic Item: {self.name.capitalize()} | Price: {self.get_price():.2f}€ | Quantity: {self.quantity} | Years of warranty: {self.warranty_years}"

    def to_dict(self):
        data = super().to_dict()
        data['warranty_years'] = self.warranty_years
        return data

# Item class, as a reference for the previous version of the project
class Item:
    """Class to represent a single product in the inventory."""
    def __init__(self, name: str, price: float, quantity: int):
        self.name = name
        self.price = price
        self.quantity = quantity

    def __str__(self):
        """Returns a string representation of the object."""
        return f"Item: {self.name.capitalize()} | Price: {self.price:.2f}€ | Quantity: {self.quantity}"

    def to_dict(self):
        """Converts the object into a dictionary for saving."""
        return self.__dict__

    @classmethod
    def from_dict(cls, data):
        """Creates an Item object from a dictionary."""
        return cls(**data)

Writing models.py


### `inventory_manager.py`

Questa cella creerà il file `inventory_manager.py` contenente la logica per la gestione dell'inventario. **Nota che importa `models.py`**, che deve essere stato creato nella cella precedente.

In [8]:
%%writefile inventory_manager.py
# inventory_manager.py

import json
from colorama import Fore, Style
from models import Item, FoodItem, ElectronicItem
from functools import wraps

# Decorator to log actions
def log_action(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        with open('actions.log', 'a') as f:
            f.write(f'{func.__name__} called with args={args[1:]}, kwargs={kwargs}\n')
        return result
    return wrapper

class InventoryManager:
    """Class that manages all operations on the inventory."""
    FILE_NAME = "inventory.json"
    
    def __init__(self):
        self.inventory = {}
        self.load_inventory()

    @log_action
    def add_item(self, item):
        if item.name in self.inventory:
            print(f"{Fore.YELLOW}Warning: Item {item.name} already exists.{Style.RESET_ALL}")
            return False
        else:
            self.inventory[item.name] = item
            print(f"{Fore.GREEN}Item {item.name} added.{Style.RESET_ALL}")
            return True

    @log_action
    def sell_item(self, name: str, sold_quantity: int):
        if name in self.inventory:
            item = self.inventory[name]
            if item.quantity >= sold_quantity:
                item.quantity -= sold_quantity
                print(f"{Fore.GREEN}{sold_quantity} {name} sold. There are left {item.quantity}.{Style.RESET_ALL}")
                return True
            else:
                print(f"{Fore.RED}Error: Not enough quantity. Ramaining: {item.quantity}{Style.RESET_ALL}")
                return False
        else:
            print(f"{Fore.RED}Error: Item {name} not exist.{Style.RESET_ALL}")
            return False

    def show_inventory(self):
        print("\n--- Current Inventory ---")
        if not self.inventory:
            print("Inventory is empty.")
        for item in self.inventory.values():
            print(item)
        print("--------------------------")
    
    def save_inventory(self):
        try:
            with open(self.FILE_NAME, 'w') as f:
                inventory_to_save = {}
                for name, item in self.inventory.items():
                    item_details = item.to_dict()
                    item_details['item_type'] = item.__class__.__name__
                    inventory_to_save[name] = item_details
                json.dump(inventory_to_save, f, indent=4)
            print(f"{Fore.GREEN}Inventory saved.{Style.RESET_ALL}")
        except IOError:
            print(f"{Fore.RED}Error saving the file.{Style.RESET_ALL}")

    def load_inventory(self):
        try:
            with open(self.FILE_NAME, 'r') as f:
                inventory_loaded = json.load(f)
                self.inventory = {}
                for name, item_details in inventory_loaded.items():
                    item_type = item_details.pop('item_type')
                    if item_type == 'FoodItem':
                        self.inventory[name] = FoodItem.from_dict(item_details)
                    elif item_type == 'ElectronicItem':
                        self.inventory[name] = ElectronicItem.from_dict(item_details)
                    elif item_type == 'Item':
                        self.inventory[name] = Item.from_dict(item_details)
            print(f"{Fore.GREEN}Inventory loaded.{Style.RESET_ALL}")
        except (IOError, json.JSONDecodeError, TypeError) as e:
            print(f"{Fore.YELLOW}No inventory existing or corrupted file. Error details: {e}{Style.RESET_ALL}")
            self.inventory = {}


Writing inventory_manager.py


### `main.py`

Questa cella creerà il file `main.py`, che contiene il loop principale del programma. **Nota che importa `inventory_manager.py` e `models.py`**.

In [9]:
%%writefile main.py
# main.py

from colorama import Fore, Style
from inventory_manager import InventoryManager
from models import Item, FoodItem, ElectronicItem

def menu():
    """Displays the menu and handles user input."""
    manager = InventoryManager()
    while True:
        manager.show_inventory()
        print("\nOptions: (a)dd, (r)emove, (s)ave, (l)oad, (q)uit")
        choice = input("Choose an option: ").lower()

        if choice == 'a':
            item_type = input("What type of item do you want to add? (f)ood, (e)lectronic or (s)imple: ").lower()
            name = input("Item name: ")
            try:
                price = float(input("Price: "))
                quantity = int(input("Quantity: "))
                if price <= 0 or quantity <= 0:
                    print(f"{Fore.RED}Error: Price and quantity must be greater than zero.{Style.RESET_ALL}")
                    continue
                
                if item_type == 'f':
                    expiration = input("Expiration date (dd-mm-yyyy): ")
                    new_item = FoodItem(name, price, quantity, expiration)
                elif item_type == 'e':
                    warranty = int(input("Warranty (years): "))
                    new_item = ElectronicItem(name, price, quantity, warranty)
                elif item_type == 's':
                    new_item = Item(name, price, quantity)
                else:
                    print(f"{Fore.RED}Invalid item type.{Style.RESET_ALL}")
                    continue
                
                manager.add_item(new_item)
            except ValueError:
                print(f"{Fore.RED}Error: Price, quantity and warranty must be valid numbers.{Style.RESET_ALL}")
        
        elif choice == 'r':
            name = input("Item name to sell: ")
            manager.sell_item(name)

        elif choice == 's':
            manager.save_inventory()
        
        elif choice == 'l':
            manager.load_inventory()

        elif choice == 'q':
            print("Thank you, program terminated.")
            break

        else:
            print(f"{Fore.RED}Invalid option, try again.{Style.RESET_ALL}")
            
if __name__ == "__main__":
    menu()

Writing main.py


---
## Esecuzione del Progetto e Simulazione 🚀

Dopo aver eseguito le celle precedenti (il che creerà i file `.py`), si può avviare il programma principale e interagire con esso. Eseguire la cella seguente per avviare il menu del programma.

In [ ]:
!python main.py

---
## Test Unitari con `unittest` 🧪

Ora che il progetto è stato implementato, è fondamentale testarne le funzionalità in modo automatico. Aggiungeremo una cartella `tests` alla struttura del progetto, dove scriveremo i nostri test unitari per la classe `GestoreInventario`.

### Preparazione della cartella di test

Eseguire la cella sottostante per creare la cartella `tests`.

In [ ]:
!mkdir -p tests

### `tests/test_inventory.py`

Questa cella creerà il file `test_inventory.py` all'interno della cartella `tests`.

In [ ]:
%%writefile tests/test_inventory.py

import unittest
import os
from inventory_manager import InventoryManager
from models import Item, FoodItem, ElectronicItem

class TestInventoryManager(unittest.TestCase):

    def setUp(self):
        # Runs before each test.
        # Ensures a clean test environment by removing the save file.
        self.manager = InventoryManager()
        if os.path.exists(self.manager.FILE_NAME):
            os.remove(self.manager.FILE_NAME)
        # Initializes a new manager that will start with an empty inventory
        self.manager = InventoryManager()
        self.test_item = Item("Test Item", 10.0, 5)

    def tearDown(self):
        # Runs after each test.
        # Cleans up by removing the save file created.
        if os.path.exists(self.manager.FILE_NAME):
            os.remove(self.manager.FILE_NAME)

    def test_add_item_success(self):
        self.assertTrue(self.manager.add_item(self.test_item))
        self.assertEqual(len(self.manager.inventory), 1)
        self.assertIn("Test Item", self.manager.inventory)

    def test_add_existing_item(self):
        self.manager.add_item(self.test_item)
        duplicate_item = Item("Test Item", 12.0, 3)
        self.assertFalse(self.manager.add_item(duplicate_item))
        self.assertEqual(self.manager.inventory['Test Item'].quantity, 5)

    def test_sell_item_success(self):
        self.manager.add_item(self.test_item)
        self.assertTrue(self.manager.sell_item("Test Item", 3))
        self.assertEqual(self.manager.inventory['Test Item'].quantity, 2)

    def test_sell_item_insufficient_quantity(self):
        self.manager.add_item(self.test_item)
        self.assertFalse(self.manager.sell_item("Test Item", 10))
        self.assertEqual(self.manager.inventory['Test Item'].quantity, 5)

    def test_sell_nonexistent_item(self):
        self.assertFalse(self.manager.sell_item("Nonexistent Item", 1))

    def test_save_and_load_inventory(self):
        food_item = FoodItem("Milk", 1.50, 10, "30-12-2025")
        electronic_item = ElectronicItem("Laptop", 1200.0, 2, 2)
        self.manager.add_item(food_item)
        self.manager.add_item(electronic_item)
        self.manager.save_inventory()

        # Create a new instance of InventoryManager to simulate a new program start
        new_manager = InventoryManager()

        # Verify that the inventory was loaded correctly
        self.assertIn("Milk", new_manager.inventory)
        self.assertIn("Laptop", new_manager.inventory)
        self.assertEqual(new_manager.inventory['Milk'].quantity, 10)
        self.assertEqual(new_manager.inventory['Laptop'].get_price(), 1200.0)
        self.assertIsInstance(new_manager.inventory['Milk'], FoodItem)
        self.assertIsInstance(new_manager.inventory['Laptop'], ElectronicItem)

if __name__ == '__main__':
    unittest.main()


### Esecuzione dei Test Unitari

Eseguire la cella seguente per avviare i test unitari. Se tutto il codice è stato scritto correttamente, si dovrebbe vedere l'output dei test che confermano il corretto funzionamento del programma.

In [ ]:
python -m unittest test_inventory.py

&copy; 2025 hanam.ai - All rights reserved. | Built with precision for real-time data streaming excellence.